In [2]:
import json
import pennylane as qml
import pennylane.numpy as np


WIRES = 2
LAYERS = 5
NUM_PARAMETERS = LAYERS * WIRES * 3

initial_params = np.random.random(NUM_PARAMETERS)

def variational_circuit(params,hamiltonian):
    parameters = params.reshape((LAYERS, WIRES, 3))
    qml.templates.StronglyEntanglingLayers(parameters, wires=range(WIRES))
    return qml.expval(qml.Hermitian(hamiltonian, wires = [0,1]))

def optimize_circuit(params,hamiltonian):

    dev = qml.device("default.qubit", wires=WIRES) # Initialize the device.

    circuit = qml.QNode(variational_circuit, dev)# Instantiate the QNode from variational_circuit.

    # Write your code to minimize the circuit
    # Define the cost function
    def cost(params):
        return circuit(params, hamiltonian)

    # Initialize the optimizer
    opt = qml.GradientDescentOptimizer(stepsize=0.1)

    # Set the number of optimization steps
    steps = 200

    # Optimization loop
    for _ in range(steps):
        params = opt.step(cost, params)

    # Return the value of the minimized QNode
    return cost(params)

# These functions are responsible for testing the solution.
def run(test_case_input: str) -> str:
       
    ins = np.array(json.loads(test_case_input), requires_grad = False)
    hamiltonian = np.array(ins,float).reshape((2 ** WIRES), (2 ** WIRES))
    np.random.seed(1967)
    initial_params = np.random.random(NUM_PARAMETERS)
    out = str(optimize_circuit(initial_params,hamiltonian))    
    
    return out


def check(solution_output: str, expected_output: str) -> None:
    solution_output = json.loads(solution_output)
    expected_output = json.loads(expected_output)
    assert np.isclose(solution_output, expected_output, rtol=5e-2)